In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("/scratch/pvg2018/Mistral7b", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("/scratch/pvg2018/Mistral7b")

prompt = "My favourite condiment is"          

model_inputs = tokenizer([prompt], return_tensors="pt")#.to("cuda")
#model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Loading checkpoint shards: 100%|██████████| 2/2 [00:18<00:00,  9.18s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> My favourite condiment is soy sauce. I like the sweet Chinese style ones the best but I also like the salty Japanese ones too. There is a special one I like and it is the Thai style fish sauce.\n\nSoy sauce is a great addition to salad dressing… you can use it to make a simple one by mixing some rice vinegar, sesame oil, chili oil, chopped ginger and a tablespoon or two of light soy sauce .\n\nAlso a good and'

In [15]:
import json

def formatting_prompts_func(example):
    text = f"SUBREDDIT: r/{example['subreddit']}\nTITLE: {example['title']}\nPOST: {example['post']}\nTL;DR: {example['summary']}"
    return text

def formatting_prompts_func_without_TLDR(example):
    text = f"SUBREDDIT: r/{example['subreddit']}\nTITLE: {example['title']}\nPOST: {example['post']}\nTL;DR:"
    return text

file_path = '/scratch/pvg2018/TLDR-Reddit/test.jsonl' 
prompt = ''
i = 0
with open(file_path, 'r') as file:
    for line in file:
        data = json.loads(line)
        if i == 2:
            prompt += formatting_prompts_func_without_TLDR(data) + '\n'
            break
        else:
            prompt += formatting_prompts_func(data) + '\n'
        i = i + 1

print(prompt)

SUBREDDIT: r/relationships
TITLE: Me [19 F] with my friend [19 M], not sure if I may have messed things up already.
POST: Hello hello everybody. I hope this isn't too trivial of a question to ask on here, but I've been feeling a bit out of my depth when it comes to this situation (I've had only one relationship before, and for many reasons, it was out of the ordinary).

Okay! So, a couple of weeks ago, I started talking to this guy on Facebook, through a student group that we were both part of. I thought he was sort of cute, so I sent him a PM just to talk, etc, etc. We're both transfer students at the same school, so I knew that we could eventually meet in person once we both moved on-campus. So, we did, and we hung out maybe twice, just as friends.

Okay. So, everything is going pretty well. We talk over Facebook and Snapchat, whatever. So, Saturday night, I was just hanging out with people and kind of being bored, when I got a Snapchat from him asking what I was doing. I asked if he

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("/scratch/pvg2018/Mistral7b", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("/scratch/pvg2018/Mistral7b")        

model_inputs = tokenizer([prompt], return_tensors="pt")#.to("cuda")
#model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

Loading checkpoint shards: 100%|██████████| 2/2 [00:16<00:00,  8.43s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> SUBREDDIT: r/relationships\nTITLE: Me [19 F] with my friend [19 M], not sure if I may have messed things up already.\nPOST: Hello hello everybody. I hope this isn\'t too trivial of a question to ask on here, but I\'ve been feeling a bit out of my depth when it comes to this situation (I\'ve had only one relationship before, and for many reasons, it was out of the ordinary).\n\nOkay! So, a couple of weeks ago, I started talking to this guy on Facebook, through a student group that we were both part of. I thought he was sort of cute, so I sent him a PM just to talk, etc, etc. We\'re both transfer students at the same school, so I knew that we could eventually meet in person once we both moved on-campus. So, we did, and we hung out maybe twice, just as friends.\n\nOkay. So, everything is going pretty well. We talk over Facebook and Snapchat, whatever. So, Saturday night, I was just hanging out with people and kind of being bored, when I got a Snapchat from him asking what I was doing

In [1]:
import json
import random

def formatting_prompts_func(example):
    return f"SUBREDDIT: r/{example['subreddit']}\nTITLE: {example['title']}\nPOST: {example['post']}\nTL;DR: {example['summary']}"

def formatting_prompts_func_without_TLDR(example):
    return f"SUBREDDIT: r/{example['subreddit']}\nTITLE: {example['title']}\nPOST: {example['post']}\nTL;DR:"

def process_file(file_path, output_file_path):
    # Read the JSONL file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Convert each line to a dictionary
    data = [json.loads(line.strip()) for line in lines]
    all_outputs = []

    # Process each row in the data
    for example in data:
        # Randomly sample 2 different rows for processing
        sampled_examples = random.sample([e for e in data if e != example], 2)
        
        # Apply formatting_prompts_func to the sampled rows
        processed_sample = [formatting_prompts_func(sample) for sample in sampled_examples]
        
        # Append the formatting_prompts_func_without_TLDR of the current row
        without_tldr_output = formatting_prompts_func_without_TLDR(example)
        
        # Collect the formatted texts
        formatted_output = "\n\n".join(processed_sample) + "\n\n" + without_tldr_output
        all_outputs.append(formatted_output)

    # Save the processed data to a new JSONL file
    with open(output_file_path, 'w') as outfile:
        for item in all_outputs:
            outfile.write(json.dumps({"processed_text": item}) + '\n')

# Specify the file paths
input_file_path = '/scratch/pvg2018/TLDR-Reddit/test.jsonl'
output_file_path = '/scratch/pvg2018/TLDR-Reddit/SummaryPrompts.jsonl'

# Process the file and save the output
process_file(input_file_path, output_file_path)

print(f"Processed data has been saved to {output_file_path}")


Processed data has been saved to /scratch/pvg2018/TLDR-Reddit/SummaryPrompts.jsonl


In [ ]:
import json
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

# Initialize the model and tokenizer
model_path = "/scratch/pvg2018/Mistral7b"
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)

def process_prompts(input_file_path, output_file_path):
    # Read the input JSONL file with prompts
    with open(input_file_path, 'r') as infile:
        lines = infile.readlines()

    results = []
    
    for line in tqdm(lines, desc="Generating responses"):
        data = json.loads(line.strip())
        prompt = data.get("processed_text", "")  # Extract the prompt from each line
        
        # Tokenize the input and generate responses
        model_inputs = tokenizer(prompt, return_tensors="pt")#.to("cuda")
        generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
        response = tokenizer.batch_decode(generated_ids)[0]
        
        # Save the prompt and response in the results
        results.append({"prompt": prompt, "response": response})
    
    # Save results to a new JSON file
    with open(output_file_path, 'w') as outfile:
        json.dump(results, outfile, indent=4)

# Paths to your files
input_file_path = '/scratch/pvg2018/TLDR-Reddit/SummaryPrompts.jsonl'
output_file_path = '/scratch/pvg2018/TLDR-Reddit/SummaryMistral7b.jsonl'

# Process the prompts and save the outputs
process_prompts(input_file_path, output_file_path)

print(f"Inferences have been saved to {output_file_path}")

/ext3/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
Generating responses:   0%|          | 2/6553 [13:29<737:05:59, 405.06s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
import json
from openai import ChatCompletion

def read_jsonl(file_path):
    with open(file_path, 'r') as file:
        return [json.loads(line.strip()) for line in file.readlines()]

def read_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

def create_gpt4_prompt(post, summary_a, summary_b):
    prompt_text = f"""
system
You are a helpful assistant, that ranks models by the quality of their answers.

user
Which of the following summaries does a better job of summarizing the most important points in the given forum post, without including unimportant or irrelevant details? A good summary is both precise and concise.
Post: "{post}"
Summary A:
{{
"model": "LLM",
"summary": "{summary_a}"
}}
Summary B:
{{
"model": "Original",
"summary": "{summary_b}"
}}
Now please rank the models by the quality of their summaries, so that the model with rank 1 has the best summary. Then return a list of the model names and ranks, i.e., produce the following output:
[
{{'model': <model-name>, 'rank': <model-rank>}},
{{'model': <model-name>, 'rank': <model-rank>}}
]
Your response must be a valid Python dictionary and should contain nothing else because we will directly execute it in Python. Please provide the ranking that the majority of humans would give.
"""
    return prompt_text

def evaluate_summaries(original_data, inference_data):
    client = ChatCompletion()
    results = []
    for original, inferred in zip(original_data, inference_data):
        prompt = create_gpt4_prompt(original['post'], inferred['response'], original['summary'])
        response = client.create(prompt=prompt, model="gpt-4", max_tokens=150)
        result = eval(response.choices[0].text)
        results.append(result)
    return results

# Paths to your files
original_file_path = '/scratch/pvg2018/TLDR-Reddit/test.jsonl'
inference_file_path = '/scratch/pvg2018/TLDR-Reddit/SummaryMistral7b.jsonl'

# Read data
original_data = read_jsonl(original_file_path)
inference_data = read_json(inference_file_path)

# Evaluate and compare summaries
comparison_results = evaluate_summaries(original_data, inference_data)

# Optionally, print or save the results
print(comparison_results)
